Скачайте датасет и проведите RFM анализ. В каждом подсегменте поделите пользователей на 4 класса. Отсчитывайте количество дней, прошедших с момента последней покупки, с максимальной даты покупки в датасете.

Необходимо ответить на вопросы:
Какое максимальное кол-во покупок было совершено одним пользователем?
Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М? (Другими словами: пользователи, у которых сумма покупок от 0 до Х попадают в 4 класс в подсегменте М)?
Какая нижняя граница у количества покупок у пользователей с классом 1 в подсегменте F?
Какая верхняя граница у количества покупок у пользователей с классом 2 в подсегменте R?
Сколько пользователей попало в сегмент 111?
Сколько пользователей попало в сегмент 311?
В каком RFM-сегменте самое большое кол-во пользователей?
В каком RFM-сегменте самое маленькое кол-во пользователей?
Какое количество пользователей попало в самый малочисленный сегмент?

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('RFM_ht_data.csv',  parse_dates=['InvoiceDate'],  low_memory=False)
#без параметра low_mmeory=False выдавалось предупреждение о смешанном типе данных

In [3]:
df.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332730 entries, 0 to 332729
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   InvoiceNo     332730 non-null  object        
 1   CustomerCode  332730 non-null  object        
 2   InvoiceDate   332730 non-null  datetime64[ns]
 3   Amount        332730 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 10.2+ MB


In [5]:
#Вычислим максимальную дату покупки в датасете
last_date = df.InvoiceDate.max()

In [6]:
last_date

Timestamp('2020-09-30 00:00:00')

In [7]:
#Создадим таблицу с RFM метриками для каждого пользователя
RFM_data = df.groupby('CustomerCode')\
.agg(Recency = ('InvoiceDate', lambda x: (last_date - x.max()).days)\
     , Frequency = ('InvoiceNo', 'nunique'), Monetary=('Amount','sum')).reset_index()

In [8]:
RFM_data

,CustomerCode,Recency,Frequency,Monetary
0,02213019,19,1,1609.20
1,02213042,22,3,9685.48
2,02213071,29,1,415.00
3,02213088,23,1,305.00
4,02213092,25,1,1412.88
...,...,...,...,...
123728,99099927,10,1,961.10
123729,99099936,0,1,1521.78
123730,99099959,8,2,1444.56
123731,99099963,19,1,3018.91


Какое максимальное кол-во покупок было совершено одним пользователем?

In [9]:
print(f' Максимальное число покупок {RFM_data.Frequency.max()}')

 Максимальное число покупок 204


In [10]:
#Продолжим находить RFM-сегменты
#Вычислим квантили
rfm_quantiles = RFM_data[['Recency', 'Monetary', 'Frequency']].quantile(q=(0.25, 0.5, 0.75))

In [11]:
rfm_quantiles

,Recency,Monetary,Frequency
0.25,2.0,765.00,1.0
0.50,8.0,1834.48,2.0
0.75,16.0,4008.84,3.0


In [12]:
#Напишем функцию, которая будет присваивать ранг, в зависимости от квантиля
#В соответствии с условием, более благоприятные показатели соответствуют более низкому рангу
def RClass(value,parameter_name, quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 3
    else:
        return 4


def FMClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 2
    else:
        return 1

In [13]:
#Добавим в датасет колонки при помощи функции
RFM_data['R_Quartile'] = RFM_data['Recency'].apply(lambda x: RClass(x, 'Recency', rfm_quantiles))

RFM_data['F_Quartile'] = RFM_data['Frequency'].apply(lambda x: FMClass (x, 'Frequency', rfm_quantiles))

RFM_data['M_Quartile'] = RFM_data['Monetary'].apply(lambda x: FMClass (x, 'Monetary', rfm_quantiles))

RFM_data['RFMClass'] = RFM_data[['R_Quartile', 'F_Quartile', 'M_Quartile']].astype(str).apply(''.join, axis=1)

In [14]:
RFM_data.head()

,CustomerCode,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMClass
0,02213019,19,1,1609.20,4,4,3,443
1,02213042,22,3,9685.48,4,2,1,421
2,02213071,29,1,415.00,4,4,4,444
3,02213088,23,1,305.00,4,4,4,444
4,02213092,25,1,1412.88,4,4,3,443


Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М? (Другими словами: пользователи, у которых сумма покупок от 0 до Х попадают в 4 класс в подсегменте М)? 

In [15]:
print(f'Ответ: {rfm_quantiles.Monetary[0.25]}')

Ответ: 765.0


Какая нижняя граница у количества покупок у пользователей с классом 1 в подсегменте F? 

In [28]:
print(f'Ответ: {rfm_quantiles.Frequency[0.75]+1}') #т.к. значение 75-го процентиля попадает в предыдущий класс

Ответ: 4.0


Какая верхняя граница у количества покупок у пользователей с классом 2 в подсегменте R? 

In [17]:
print(f'Ответ: {rfm_quantiles.Recency[0.50]}')

Ответ: 8.0


Сколько пользователей попало в сегмент 111? 

In [18]:
one_one_one = RFM_data.query("RFMClass == '111'").RFMClass.size

In [19]:
print(f'Ответ: {one_one_one}')

Ответ: 9705


Сколько пользователей попало в сегмент 311? 

In [20]:
three_one_one = RFM_data.query("RFMClass == '311'").RFMClass.size

In [21]:
print(f'Ответ: {three_one_one}')

Ответ: 1609


В каком RFM-сегменте самое большое кол-во пользователей?

In [22]:
#Посчитаем число пользователей в сегментах
RFM_number = RFM_data.RFMClass.value_counts().reset_index().rename(columns={'index':'RFMClass','RFMClass':'Number'})

In [23]:
RFM_descending = RFM_number.sort_values('Number', ascending = False)

In [24]:
print(f'Ответ: {RFM_descending.RFMClass[0]}') 

Ответ: 444


В каком RFM-сегменте самое маленькое кол-во пользователей? 

In [25]:
RFM_ascending = RFM_number.sort_values('Number').reset_index()

In [26]:
print(f'Ответ: {RFM_ascending.RFMClass[0]}') 

Ответ: 414


Какое количество пользователей попало в самый малочисленный сегмент?

In [27]:
print(f'Ответ: {RFM_ascending.Number[0]}') 

Ответ: 2
